# Поиск футбольных логотипов для городов России

**Цель**: получить для разных городов России логотипы местных футбольных команд.


**Задачи**:

- Запросить информацию о всех футбольных клубах России, включая их логотипы и города базирования из Wikidata.

- Для тех клубов, для которых нет логтипа на Wikidata (большинство), найти недостающие на странице Wikipedia.

- Очистить данные, оставив в выборке только футбольные команды ассоциированные с городами и у которых есть логотип.

## Импорт библиотек

In [173]:
import requests
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
from time import sleep
from tqdm import tqdm

## Парсинг данных

In [165]:
def fetch_wikidata_football_clubs():
    """
    Запрашивает у Wikidata информацию по футбольным клубам России.
    Название команды, логотип, город и название статьи в российской википедии, если такая есть.
    """
    endpoint = "https://query.wikidata.org/sparql"
    query = """
    SELECT DISTINCT ?team ?teamLabel ?wikiPage ?logo ?hqLocationLabel ?countryLabel WHERE {
      ?team wdt:P31 wd:Q476028;                # Instance of football club
            wdt:P17 wd:Q159;                   # Located in Russia (Russia = Q159)
            wdt:P17 ?country.
      OPTIONAL { ?team wdt:P154 ?logo. }       # Logo (if exists)
      OPTIONAL { ?team wdt:P159 ?hqLocation. } # Headquarters location (city)
      OPTIONAL {
          ?wikipediaArticle schema:about ?team ;
                            schema:isPartOf <https://ru.wikipedia.org/> ;
                            schema:name ?wikiPage.  # Russian Wikipedia page title
      }
      SERVICE wikibase:label { bd:serviceParam wikibase:language "ru". }
    }
    """
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    results = sparql.query().convert()
    teams = []

    for result in results["results"]["bindings"]:
        team = {
            "teamLabel": result["teamLabel"]["value"],
            "logo": result.get("logo", {}).get("value", None),  # Optional logo
            "city": result.get("hqLocationLabel", {}).get("value", None),
            "country": result.get("countryLabel", {}).get("value", None),            
            "wikiPage": result.get("wikiPage", {}).get("value", None),
        }
        teams.append(team)

    return teams

In [212]:
def fetch_wikipedia_logo(wiki_page):
    """
    Ищет на странице футбольной команды в Википедии логотип.
    """
    endpoint = "https://ru.wikipedia.org/w/api.php"
    params = {
        "action": "parse",
        "page": wiki_page,
        "prop": "images",
        "format": "json",
    }

    response = requests.get(endpoint, params=params)
    if response.status_code == 200:
        data = response.json()
        images = data.get("parse", {}).get("images", [])
        logo_images = [img for img in images if 'logo' in img.lower() and not 'Notification-icon-Commons-logo' in img]  # Filter images containing 'logo'
        if logo_images:
            return logo_images[0]  # Return the first logo image
    else:
        print(f"Failed to fetch Wikipedia page for {wiki_page}")
        return None

In [213]:
# Шаг 1: Получаем данные о футбольных командах из Wikidata
clubs = fetch_wikidata_football_clubs()

In [214]:
print(clubs[0])

{'teamLabel': 'Булат (футбольный клуб, Череповец)', 'logo': None, 'city': None, 'country': 'Россия', 'wikiPage': 'Булат (футбольный клуб, Череповец)'}


Всего клубов:

In [215]:
len(clubs)

554

In [216]:
# Шаг 2: Получаем недостающие логотипы команд из Wikipedia

for club in tqdm(clubs):
    if not club["logo"]:
        wiki_page_title = club["wikiPage"]
        sleep(0.2)
        img = fetch_wikipedia_logo(wiki_page_title)
        if img:
            club["logo"] = f'https://ru.wikipedia.org/wiki/Special:FilePath/{img}'

100%|█████████████████████████████████████████| 554/554 [05:35<00:00,  1.65it/s]


In [217]:
# Преобразуем в датафрейм
df_clubs = pd.DataFrame(clubs)
df_clubs.sample(10)

,teamLabel,logo,city,country,wikiPage
40,Галакс (футбольный клуб),None,None,Россия,Галакс
404,Горняк,None,None,Россия,"Горняк (футбольный клуб, Качканар)"
509,Локомотив-КМВ,None,None,Россия,"Локомотив (футбольный клуб, Минеральные Воды)"
296,Акрон-2,https://ru.wikipedia.org/wiki/Special:FilePath...,None,Россия,Акрон-2
277,Красный-СГАФКСТ,https://ru.wikipedia.org/wiki/Special:FilePath...,Смоленск,Россия,Красный (футбольный клуб)
313,Дагдизель,https://ru.wikipedia.org/wiki/Special:FilePath...,None,Россия,Дагдизель (футбольный клуб)
235,Красногвардеец (футбольный клуб),None,None,Россия,Красногвардеец (футбольный клуб)
492,"Диана (футбольный клуб, Волжск)",None,None,Россия,Диана (футбольный клуб)
57,СКД (футбольный клуб),None,None,Россия,СКД (футбольный клуб)
272,Содовик,https://ru.wikipedia.org/wiki/Special:FilePath...,Стерлитамак,Россия,Содовик (футбольный клуб)


Количество команд без логотипов

In [218]:
df_clubs['logo'].notna().sum()

232

In [219]:
# Убираем команды без логотипов и без городов
df_clubs = df_clubs[~df_clubs['logo'].isna() & ~df_clubs['city'].isna()]
df_clubs.sample(10)

,teamLabel,logo,city,country,wikiPage
124,"Химик (футбольный клуб, Дзержинск)",https://ru.wikipedia.org/wiki/Special:FilePath...,Дзержинск,Россия,"Химик (футбольный клуб, Дзержинск)"
100,Севастополь,https://ru.wikipedia.org/wiki/Special:FilePath...,Севастополь,Россия,"Севастополь (футбольный клуб, 2014)"
392,Кубань,https://ru.wikipedia.org/wiki/Special:FilePath...,Краснодар,Россия,Кубань (футбольный клуб)
272,Содовик,https://ru.wikipedia.org/wiki/Special:FilePath...,Стерлитамак,Россия,Содовик (футбольный клуб)
266,Коломна,https://ru.wikipedia.org/wiki/Special:FilePath...,Коломна,Россия,Коломна (футбольный клуб)
366,Лада-Тольятти,https://ru.wikipedia.org/wiki/Special:FilePath...,Тольятти,Россия,Лада-Тольятти
170,Текстильщик,https://ru.wikipedia.org/wiki/Special:FilePath...,Иваново,Россия,"Текстильщик (футбольный клуб, Иваново)"
335,Челябинск,https://ru.wikipedia.org/wiki/Special:FilePath...,Челябинск,Россия,Челябинск (футбольный клуб)
7,Волочанин-Ратмир,https://ru.wikipedia.org/wiki/Special:FilePath...,Вышний Волочёк,Россия,Волочанин
164,Ижевск,https://ru.wikipedia.org/wiki/Special:FilePath...,Ижевск,Россия,Ижевск (футбольный клуб)


Количество клубов

In [223]:
len(df_clubs['city'])

147

Охват по городам

In [220]:
df_clubs['city'].nunique()

101

Распределение по городам

In [221]:
df_clubs['city'].value_counts().head(10)

Москва             14
Краснодар           6
Санкт-Петербург     4
Нижний Новгород     4
Тольятти            3
Тамбов              3
Ижевск              3
Махачкала           3
Смоленск            2
Ставрополь          2
Name: city, dtype: int64

In [222]:
# Выведем итоговый список городов с логотипами команд
for club in df_clubs.iterrows():
    print(club[1]['city'] + ', ' + club[1]['teamLabel'] + ', ' + club[1]['logo'])

Волгодонск, Атоммаш (футбольный клуб), https://ru.wikipedia.org/wiki/Special:FilePath/FC_Atommash_Logo.svg
Вышний Волочёк, Волочанин-Ратмир, https://ru.wikipedia.org/wiki/Special:FilePath/Logo_volochanin-ratmir_vyshnij_voloch.gif
Саратов, Сокол, https://ru.wikipedia.org/wiki/Special:FilePath/FC_Sokol_Saratov_Logo.svg
Крымск, Витязь (футбольный клуб, Крымск), https://ru.wikipedia.org/wiki/Special:FilePath/Vityaz_Krymsk_logo.png
Омск, Иртыш, https://ru.wikipedia.org/wiki/Special:FilePath/FC_Irtysh_Omsk_Logo.svg
Краснодар, Краснодар-2 (футбольный клуб), https://ru.wikipedia.org/wiki/Special:FilePath/FC_Krasnodar_Logo.svg
Тихвин, Тосно, https://ru.wikipedia.org/wiki/Special:FilePath/FC_Tosno_Logo.svg
Белгород, Салют, https://ru.wikipedia.org/wiki/Special:FilePath/FC_Salyut_Belgorod_Logo.svg
Воронеж, Факел, https://ru.wikipedia.org/wiki/Special:FilePath/FC_Fakel_Voronezh_logo_2022.svg
Чита, Чита, https://ru.wikipedia.org/wiki/Special:FilePath/Logo_of_FC_Chita_(2005).gif
Афипский, Афипс, htt

## Вывод

Из данных Wikidata и Wikipedia Мы получиили данные о 554 футбольных командах в России. Оформленным (с логотипом и названием города) оказалось 147 клубов из 101 города. Из которых 14 приходится на Москву, 6 на Краснодар, 4 на Санкт-Петербург, 4 на Нижний Новгород, для остальных городов количество команд меньше.